# Join the Dots - Part A.
For a overview guide to what is happening here, please visit: https://youtu.be/Smw3suzynho

In [ ]:
##Functions and dependencies
%matplotlib inline

import pylab as plt
import tifffile
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
from ijroi.ij_roi import Roi
from ijroi.ijpython_decoder import decode_ij_roi
import keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
import imageio
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.etree import ElementTree
from xml.dom import minidom

def compare_dist(dist_x0,dist_y0,dist_x1,dist_y1):

    dist_x0 = dist_x0.astype(np.float32)
    dist_y0 = dist_y0.astype(np.float32)
    dist_x1 = dist_x1.astype(np.float32)
    dist_y1 = dist_y1.astype(np.float32)
   
    m_x0 = np.tile(dist_x0, (dist_x1.shape[0],1)).astype(np.float)
    m_x1 = np.tile(dist_x1, (dist_x0.shape[0],1)).astype(np.float)
    m_y0 = np.tile(dist_y0, (dist_y1.shape[0],1)).astype(np.float)
    m_y1 = np.tile(dist_y1, (dist_y0.shape[0],1)).astype(np.float)
    m_x2 = m_x0-m_x1.T; m_y2 = m_y0-m_y1.T

    dist = np.sqrt(m_x2**2+m_y2**2)
    mlim = np.min(dist.shape)
    dist[np.arange(0,mlim),np.arange(0,mlim)] = 9999

    return dist

def return_regs_from_img_and_sub(input_path, filename,onlyfile_sections,plot_img=False):
    regions = []


    output_secs =[]
    #opens large image.
    output, names,img = decode_roi(input_path+filename)
    if plot_img == True:
        plt.figure(figsize=(16,16))
        plt.imshow(img)
        for out in output:
            x = out[0]
            y = out[1]
            w = out[2]
            h = out[3]
            plt.plot([x,x+w,x+w,x,x],[y,y,y+h,y+h,y],'b')
            regions.append([x,y,w,h,"region"])
    else:
        for out in output:
            x = out[0]
            y = out[1]
            w = out[2]
            h = out[3]
            regions.append([x,y,w,h,"region"])


    output_index = []
    output_coords = []
    #Goes through small sections.
    for sections in onlyfile_sections:
        #If section is in filename.
        #print(sections)
        if filename == sections[0:filename.__len__()]:
            #Decode roi.
            outs, names_sects,null = decode_roi(input_path_sections+sections)

            seq_num = []
            seq_x = []

            for out,name in zip(outs,names_sects):
                x = out[0]+float(sections.split("_")[4])-null.shape[1]//2
                y = out[1]+float(sections.split("_")[5].split(".")[0])-null.shape[0]//2
                w = out[2]
                h = out[3]
                n = name
                if n == 'f':
                    seq_num =[]
                    seq_x =[]
                    if plot_img == True:
                        plt.plot([x,x+w,x+w,x,x],[y,y,y+h,y+h,y],'r')
                    continue;
                if plot_img == True:
                    plt.plot([x,x+w,x+w,x,x],[y,y,y+h,y+h,y],'g')
                regions.append([x,y,w,h,name])
                try:

                    seq_num.append(int(n))
                    seq_x.append(x)
                except:
                    dotx = x
                    doty = y

            indi = np.argsort(np.array(seq_x))
            sortd = np.array(seq_num)[indi]
            try:
                output_index.append(np.array("".join(sortd.astype(str))).astype(np.int))
                output_coords.append([dotx,doty])
            except:
                pass
    indi = np.argsort(np.array(output_index))
    sortd_index = np.array(output_index)[indi]
    sortd_coords = np.array(output_coords)[indi]
    stt_ind = 0
    end_ind = 1
    for i in range(1,sortd_index.shape[0]):
        if sortd_index[i]-sortd_index[i-1] < 3:
            continue
        else:
            end_ind = np.copy(i)
            #plt.plot(sortd_coords[stt_ind:end_ind,0],sortd_coords[stt_ind:end_ind,1],'r')
            stt_ind = np.copy(end_ind)
    #plt.plot(sortd_coords[stt_ind:i,0],sortd_coords[stt_ind:i,1],'r')
    return img, regions
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")
def decode_roi(filepath):
    overlay_arr = []
    names = []
    output = []

    if filepath.split(".")[-1]== "tif":
        tfile = tifffile.TiffFile(filepath)
        img = tfile.asarray()
        img_shape = tfile.asarray().shape[0:2]

        if 'Overlays' in tfile.imagej_metadata:
            overlays = tfile.imagej_metadata['Overlays']
            if overlays.__class__.__name__ == 'list':
                #Multiple overlays and so iterate.
                for overlay in overlays:

                    overlay_arr.append(decode_ij_roi(overlay,img_shape))
        #plt.imshow(img)
        for i in range(0,overlay_arr.__len__()):

            if overlay_arr[i] != False:
                x0 = int(overlay_arr[i].x)
                y0 = int(overlay_arr[i].y)
                wid = int(overlay_arr[i].width)
                hei = int(overlay_arr[i].height)
                name = overlay_arr[i].name
                #plt.plot([x0,x0+wid,x0+wid,x0,x0],[y0,y0,y0+hei,y0+hei,y0])
                output.append([x0,y0,wid,hei])
                names.append(name.replace('\x00',''))
        return output, names,img
class Graph: 
      
    # init function to declare class variables 
    def __init__(self,V): 
        self.V = V 
        self.adj = [[] for i in range(V)] 
  
    def DFSUtil(self, temp, v, visited): 
  
        # Mark the current vertex as visited 
        visited[v] = True
  
        # Store the vertex to list 
        temp.append(v) 
  
        # Repeat for all vertices adjacent f
        # to this vertex v 
        for i in self.adj[v]: 
            if visited[i] == False: 
                  
                # Update the list 
                temp = self.DFSUtil(temp, i, visited) 
        return temp 
  
    # method to add an undirected edge 
    def addEdge(self, v, w): 
        self.adj[v].append(w) 
        self.adj[w].append(v) 
  
    # Method to retrieve connected components 
    # in an undirected graph 
    def connectedComponents(self): 
        visited = [] 
        cc = [] 
        for i in range(self.V): 
            visited.append(False) 
        for v in range(self.V): 
            if visited[v] == False: 
                temp = [] 
                cc.append(self.DFSUtil(temp, v, visited)) 
        return cc 
def generate_xml(xml_file, img_shape, passed_regs):

    top = Element('annotation')
    folder = SubElement(top, 'folder')
    folder.text = 'dc2020'
    filename = SubElement(top, 'filename')
    filename.text = xml_file.split("/")[-1]

    source = SubElement(top, 'source')
    database = SubElement(source,'database')
    database.text = "The 2020 dot-to-dot"
    annotation = SubElement(source,'annotation')
    annotation.text = "Waithe 2020"
    image = SubElement(source,'image')
    image.text = "photo"
    size = SubElement(top, 'size')
    width = SubElement(size,'width')
    width.text = str(img_shape[1])
    height = SubElement(size,'height')
    height.text = str(img_shape[0])
    depth = SubElement(size,'depth')
    depth.text = str(img_shape[2])
    
    for reg in passed_regs:
        objects = SubElement(top, 'object')
        names = SubElement(objects,'name')
        names.text = reg[4]
        pose = SubElement(objects,'pose')
        pose.text = "Unspecified"
        truncated = SubElement(objects,'truncated')
        truncated.text = "0"
        difficult = SubElement(objects,'difficult')
        difficult.text = "0"
        bndbox = SubElement(objects, 'bndbox')
        xmin = SubElement(bndbox,"xmin")
        xmin.text = str(int(reg[0]))
        ymin = SubElement(bndbox,"ymin")
        ymin.text = str(int(reg[1]))
        xmax = SubElement(bndbox,"xmax")
        xmax.text = str(int(reg[0]+reg[2]))
        ymax = SubElement(bndbox,"ymax")
        ymax.text = str(int(reg[1]+reg[3]))

    f = open(xml_file,'w')
    f.write(prettify(top))
    f.close()
    

    

def save_regions_pascal(img,regions,net_input,bin_factor,output_file_img_root,output_file_xml_root,count,plot_on=False):
    img_ = img[::bin_factor,::bin_factor,:]
    for c in range(0,img_.shape[0]-net_input,net_input):
        for r in range(0,img_.shape[1]-net_input,net_input):
            im_ = img_[c:c+net_input,r:r+net_input,:]
            if plot_on == True:
                plt.figure()
                plt.imshow(im_)
            passed = 0
            passed_regs = []
            for reg in regions:
                x = reg[0]//bin_factor
                y = reg[1]//bin_factor
                w = reg[2]//bin_factor
                h = reg[3]//bin_factor
                n = reg[4]
                if y > c and y < c+net_input and x > r and x < r+net_input:
                    if n == 'region':
                        passed +=1
                        passed_regs.append([x-r,y-c,w,h,n])
                        if plot_on == True:
                            plt.plot([x-r,x+w-r,x+w-r,x-r,x-r],[y-c,y-c,y+h-c,y+h-c,y-c],'r')
            if passed > 0:
                count += 1
                n = str(count)
                cnt = n.zfill(6)
                imageio.imsave(output_file_img_root+cnt+".jpg",im_)
                generate_xml(output_file_xml_root+cnt+".xml",im_.shape,passed_regs)
    print("total_count",count)
    print("excessc",img_.shape[0]-(c+net_input))
    print("excessr",img_.shape[1]-(r+net_input))
    return count
def save_regions_EAST(img,regions,net_input,bin_factor,output_file_img_root,count,plot_on=False):
    img_ = img[::bin_factor,::bin_factor,:]
    for c in range(0,img_.shape[0]-net_input,net_input):
        for r in range(0,img_.shape[1]-net_input,net_input):
            im_ = img_[c:c+net_input,r:r+net_input,:]
            if plot_on == True:
                plt.figure()
                plt.imshow(im_)
            passed = 0
            passed_regs = []
            for reg in regions:
                
                x = reg[0]//bin_factor
                y = reg[1]//bin_factor
                w = reg[2]//bin_factor
                h = reg[3]//bin_factor
                n = reg[4]
                if y > c and y+h < c+net_input and x > r and x+w < r+net_input:
                    if n == 'region':
                        passed +=1
                        passed_regs.append([x-r,y-c,w,h,n])
                        if plot_on == True:
                            plt.plot([x-r,x+w-r,x+w-r,x-r,x-r],[y-c,y-c,y+h-c,y+h-c,y-c],'r')
            if passed > 0:
                count += 1
                n = str(count)
                cnt = n.zfill(6)
                imageio.imsave(output_file_img_root+cnt+".jpg",im_)
                f = open(output_file_img_root+cnt+".txt",'w')
                c = 0
                for reg in passed_regs:
                    line_str = ""
                    line_str += str(reg[0]) + ","
                    line_str += str(reg[1]) + ","
                    line_str += str(reg[0]+reg[2]) + ","
                    line_str += str(reg[1]) + ","
                    line_str += str(reg[0]+reg[2]) + ","
                    line_str += str(reg[1]+reg[3]) + ","
                    line_str += str(reg[0]) + ","
                    line_str += str(reg[1]+reg[3]) + ","
                    line_str += reg[4]
                    if c< passed_regs.__len__()-1:
                        line_str += "\n"
                    f.write(line_str)
                    c+=1
                f.close()
    print("total_count",count)
    print("excessc",img_.shape[0]-(c+net_input))
    print("excessr",img_.shape[1]-(r+net_input))
    return count
def resize_it(imi, shape):
    laxis = np.max(imi.shape)
    padw = (laxis-imi.shape[0])/2
    padh = (laxis-imi.shape[1])/2

    imb = np.pad(imi[:,:,0], ((int(np.floor(padw)),int(np.ceil(padw))),(int(np.floor(padh)),int(np.ceil(padh)))), 'minimum')

    return np.array(Image.fromarray(imb).resize(shape))
def rtn_valid_dots(cc,labs,ans,regions):
    dot2dot = []
    idi = 0
    for cc0 in cc:
        nh = []
        pasd = False
        d = None
        for b in cc0:
            
            if ans[labs[b]] == 'f':
                #Any fails is not a good sign. Lets leave it.
                pasd = False;
                break;
            if ans[labs[b]] == 'dot' or ans[labs[b]] == 'star' or ans[labs[b]] == 'triangle' :
                pasd = True;
                d = np.copy(b)
            
            nh.append([regions[b],d])
        if nh.__len__() ==1:
            continue
        if pasd == True:
            #Our cluster has passed.

            order = []
            number = []
            #Now we need to intrepret the number.
            for nh0,d in nh:
                if nh0[4] == 'dot' or nh0[4] == 'star' or nh0[4] == 'triangle' :
                 c_x = nh0[0]+nh0[2]//2
                 c_y = nh0[1]+nh0[3]//2
                else:
                    order.append(nh0[0])
                    number.append(nh0[4])
            #print(order,number)  
            if number !=[]:
                indi = np.argsort(order)
                
                dot2dot.append([int("".join(np.array(number)[indi])),c_x,c_y,idi,d])
            
        idi +=1
    if dot2dot ==[]:
        return []
    dot2dot = np.array(dot2dot)
    
    return dot2dot[np.argsort(np.array(dot2dot)[:,0]),:]
def rtn_regions(img,plot=False):
    gray = 255-cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)
    blur = cv2.GaussianBlur(gray,(61,61),30)
    filt = gray-blur
    filt[filt<0] = 0
    
    ret3,th3 = cv2.threshold(filt.astype(np.uint8),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)  
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(th3, connectivity=4)
    
    if plot == True:
       
        plt.figure(figsize=(132,16))
    regions = []
    for reg in stats[1:]:
        x = int(reg[0])
        y = int(reg[1])
        w = int(reg[2])
        h = int(reg[3])
        
        regions.append([x,y,w,h,'region'])
        if w*h > 40:
            
            if plot == True:
                plt.plot([x,x+w,x+w,x,x],[y,y,y+h,y+h,y],'-',lw=1)

        
    if plot == True:
        plt.imshow(img)
        for reg in regions:
            w = reg[2]
            h = reg[3]
            x = reg[0]
            y = reg[1]
            plt.plot([x,x+w,x+w,x,x],[y,y,y+h,y+h,y],'-',lw=1)
    return regions,th3

def ccw(A,B,C):
        return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])


def line_intersect(line0,line1):
    # Return true if line segments AB and CD intersect
    (A,B) = line0
    (C,D) = line1
    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

## Imports the training material from folders.

In [ ]:
#Import numbers from regions.
#Training path.
input_path_sections = "images/sections_labels/"
onlyfile_sections = [f for f in listdir(input_path_sections) if isfile(join(input_path_sections, f))]
letters = []
labels  =[] 
for filename in onlyfile_sections:
    if filename != ".DS_Store":
        output, names,img = decode_roi(input_path_sections+filename)
        if output != []:
            for out, name in zip(output,names):
                #if name != "f":
                    imi = 255-img[out[1]:out[1]+out[3],out[0]:out[0]+out[2]]
                    laxis = np.max(imi.shape)
                    padw = (laxis-imi.shape[0])/2
                    padh = (laxis-imi.shape[1])/2
            
                    imb = np.pad(imi[:,:,0], ((int(np.floor(padw)),int(np.ceil(padw))),(int(np.floor(padh)),int(np.ceil(padh)))), 'minimum')
                
                    im = np.array(Image.fromarray(imb).resize((28,28)))
                    letters.append(im)
                    labels.append(name)
print('data imported')

## Import data to train mnist style classifier

In [ ]:
from keras.models import Sequential


labels_cat = []
for lab in labels:
    if lab == 'dot':
        labels_cat.append('10')
    elif lab == "square":
        labels_cat.append('10')
    elif lab == "star":
        labels_cat.append('10')
    elif lab == "triangle":
        labels_cat.append('10')
    elif lab == "f":
        labels_cat.append('11')
    else:
        labels_cat.append(lab)


# Split the data
X_train, X_test, y_train, y_test = train_test_split(np.array(letters), np.array(labels_cat), test_size=0.33, shuffle= True)

print(X_train.shape)
#reshaping
#this assumes our data format
#For 3D data, "channels_last" assumes (conv_dim1, conv_dim2, conv_dim3, channels) while 
#"channels_first" assumes (channels, conv_dim1, conv_dim2, conv_dim3).
img_rows = 28
img_cols = 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape) #X_train shape: (60000, 28, 28, 1)
#set number of categories
num_category = 12
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_category)
y_test = keras.utils.to_categorical(y_test, num_category)


##model building

num_category = 12
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
model.add(Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
model.add(Dense(num_category, activation='softmax'))


#Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
#categorical ce since we have multiple classes (10) 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
batch_size = 128
num_epoch = 20
#model training
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

#Can be used to export images used for testing
#c = 0
#for im in X_test:
#    cv2.imwrite("examples/"+str(c)+".png",im*255)
#    c+=1

## The dot-2-dot script which does the work

In [ ]:

#Evaluate frame of movies.
import time
import cv2
ans = ['0','1','2','3','4','5','6','7','8','9','dot','f']
colours = np.random.rand(32,3) #used only for display

input_path = "/Users/dwaithe/Documents/youtube_videos/computerVisionProjects/join_the_dots/images/tiffsInputs/"
output_path = "/Users/dwaithe/Documents/youtube_videos/computerVisionProjects/join_the_dots/images/scanned_with_lines/"    

for t in range(45,46):
    
    plot_on = False
    t1 = time.time()
    n= str(t).zfill(3)
    
    filename = "IMG_00"+str(t)+".tif"
    filename = "pc5bbobdi.tiff"
    img = cv2.imread(input_path +filename)
    
    #This is the main feature finding function.
    regions, filt = rtn_regions(img,plot=plot_on)
    
    
    #Here we initiate some things.
    ttl_arr =[] #array to hold small image patches of our letters
    dist_x = np.zeros((regions.__len__())).astype(np.int32)
    dist_y = np.zeros((regions.__len__())).astype(np.int32)
    c = 0

    for reg in regions:
        x,y,w,h,s = reg
        #we populate our coordinate matrixes
        dist_x[c] = x
        dist_y[c] = y
        ttl_arr.append(resize_it(255-img[y:y+h,x:x+w,:],(28,28)))
        c+=1

    probs = model.predict(np.array(ttl_arr).reshape(regions.__len__(),28,28,1))
    labs = np.argmax(probs,1)
    t2 = time.time()
    if plot_on:
        plt.figure(figsize = (16,16))
        #plt.ylim(1750,1200)
        plt.imshow(img)  
    
    for rgx in np.arange(0,regions.__len__()):
        regions[rgx][4] = ans[labs[rgx]]
        reg = regions[rgx]
        x = int(reg[0])
        y = int(reg[1])
        w = int(reg[2])
        h = int(reg[3])
        
        #if reg[4] == 'dot':
        #    image = cv2.circle(img,(int(x),int(y)),5,(255,255,255),5)
        #if plot_on:
         #plt.text(x,y,str(ans[labs[rgx]]),c='r')
  
    t0 = time.time()
    X = np.zeros((regions.__len__(),4))

    for ct,reg in enumerate(regions):
        X[ct,0] = reg[0]+reg[2]//2
        X[ct,1] = reg[1]+reg[3]//2
        

    #Comparing all the non-dots.
    dist_dot_dot = compare_dist(X[labs!=10,0],X[labs!=10,1],X[labs!=10,0],X[labs!=10,1])       
    lisl = np.where(labs!=10)[0]
    pairs =[]
    for ct,lab in enumerate(np.where(labs!=10)[0]):
        distt = dist_dot_dot[:,ct]
        nn = np.argmin(distt)
        if np.min(distt)<50:
            pairs.append([lisl[ct],lisl[nn]])


    #Generate graph to find neighbourhoods.
    g = Graph(regions.__len__()); 
    for nnn in pairs:
        g.addEdge(nnn[0], nnn[1])
        

    #Now we find the nearest dot to each non-dot.
    dist_dot_ndot = compare_dist(X[labs==10,0],X[labs==10,1],X[labs!=10,0],X[labs!=10,1])

    #nondots vs dots distance matrix calcualtion
    lisl = np.where(labs==10)[0]
    lisnl = np.where(labs!=10)[0]
    pairs =[]
    for ct,lab in enumerate(np.where(labs==10)[0]):
        distt = dist_dot_ndot[:,ct]
        nn = np.argmin(distt)
        if np.min(distt)<60:
            pairs.append([lisl[ct],lisnl[nn]])


    #plot the connections between regions.
    for nnn in pairs:
        g.addEdge(nnn[0], nnn[1]) 
    cc = g.connectedComponents()

  
    #Returns valid dots.
    dot2dot = rtn_valid_dots(cc,labs,ans,regions)
    dts = np.zeros((X.shape[0]))
    if dot2dot != []:
        dts[dot2dot[:,4]] = dot2dot[:,0]
    dets = np.array((X[labs==10,0],X[labs==10,1],np.array(dts[labs==10]))).T
    
    order = np.argsort(dets[:,2])
    reorgt = dets[order,:]
    difv = np.diff(reorgt[:,2])
    line_segments = []
    for i in range(0,difv.shape[0]):
        if difv[i] >0 and difv[i]<4:
            if np.sqrt((reorgt[i,0]-reorgt[i+1,0])**2 + (reorgt[i,1]-reorgt[i+1,1])**2):# <150000:
                start_point = (int(reorgt[i,0]),int(reorgt[i,1]))
                end_point =  (int(reorgt[i+1,0]),int(reorgt[i+1,1]))
                
                line_segments.append([start_point,end_point])
        
                
    if plot_on:
        plt.figure(figsize=(16,16))
        plt.imshow(img)
    #This is a filter to improve the quality of the output. Line segments shouldn't really overlap.
    #If a line segment overlaps more than a few times with other line segments, it should be voided as it is likely wrong.
    f = open(output_path+filename.split(".")[-2]+'.txt','w')
    for l0 in range(0,line_segments.__len__()):
        line0 = line_segments[l0]
        count_intersect = 0
        for l1 in range(0,line_segments.__len__()):
            line1 = line_segments[l1]
            if l0 != l1:
                if line_intersect(line0,line1):
                    count_intersect +=1
        #This is the number of intersections that a line must make for it to be banned.
        #if count_intersect < 5:
        if plot_on:
            plt.plot([line0[0][0],line0[1][0]],[line0[0][1],line0[1][1]],'b-')
        color = (0, 0, 255)            
        img = cv2.line(img, line0[0], line0[1], color, 8)
        f.writelines(str(line0[0])+","+str(line0[1])+'\n')
            
   
    imRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_path+filename.split(".")[-2]+'.png',imRGB)
    f.close()
    print('time (ms)',time.time()-t1)